In [ ]:
import reproducibility

In [ ]:
# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
from keras.utils.vis_utils import plot_model
from scipy.interpolate import LinearNDInterpolator, interpn
from scipy.optimize import root
from utils import hash2

# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_raws, retrieve_raws, format_precip, fixnan
from data_funcs import raws_data, synthetic_data, plot_data, check_data, mse_data, to_json, from_json
import moisture_models as mod
from moisture_rnn import create_RNN_2, staircase, create_rnn_data, train_rnn, rnn_predict

meso_token="b40cb52cbdef43ef81329b84e8fd874f"


In [ ]:
# Change directory for data read/write
import os
os.chdir('data')

## Retrieve RAWS Data

In [ ]:
# raws_dat=raws_data(start='201806010800', hours=1200, h2=300, stid="BKCU1",meso_token=meso_token)
raws_dat=from_json('kf_orig.json')
# hash2(raws_dat)

In [ ]:
%matplotlib inline
plot_data(raws_dat)

In [ ]:
plot_data(raws_dat,hmax=600)

In [ ]:
raws_dat.keys()

In [ ]:
print('fm hash:', hash2(raws_dat['fm']))
print('rain hash:', hash2(raws_dat['rain']))
print('Ed hash:', hash2(raws_dat['Ed']))
print('Ew hash:', hash2(raws_dat['Ew']))
print('m hash:', hash2(raws_dat['m']))

## Interface
Jonathon changes above  create each case as a dictionary, then dictionary of dictionaries, figure out how to store and load dictionaries as a file. json is possible but: cannot contain datetime objects
look into pickle also compresses while json is plain text clone wrfxpy look how for idioms, pickle added jan/angel lager
Jan will edit from here below. 
cases will be extracted from dictionary as global variables for now at least

In [ ]:
np.random.seed(seed=123)

In [ ]:
# dictionary raws_dat has all that is needed for the run 
# keeping the name raws_dat for now even if it may not be raws data



In [ ]:
synt_dat=synthetic_data()  # just testinh
%matplotlib inline
plot_data(synt_dat)

## Fit Augmented KF

In [ ]:
raws_dat.keys()

In [ ]:
m,Ec = mod.run_augmented_kf(raws_dat)  # extract from state
raws_dat['m']=m
raws_dat['Ec']=Ec
plot_data(raws_dat,title2='augmented KF')

In [ ]:
plot_data(raws_dat,hmin=0,hmax=600)

In [ ]:
plot_data(raws_dat,hmin=900,hmax=1200,title2='augmented KF prediction detail')

In [ ]:
mse_data(raws_dat)


## Fit RNN Model

In [ ]:
# Set seed for reproducibility
tf.random.set_seed(123)

In [ ]:
import pickle
# case_data = from_json('rnn_orig.json')
with open('testing_dict.pickle', 'rb') as handle:
    test_dict = pickle.load(handle)
case_data = test_dict['case11']

In [ ]:
plot_data(case_data,title2=' from rnn_orig.json',hmin=0,hmax=600)

In [ ]:
plot_data(case_data,title2='RNN prediction',hmin=300,hmax=600)

In [ ]:
if 'm' in case_data:
    mse_data(case_data)  # just check sdolution if there
    del case_data['m']   # cleanup - remove old solution if any

In [ ]:
verbose = False
# Set seed for reproducibility
tf.random.set_seed(123)
rnn_dat = create_rnn_data(case_data,scale=False, verbose=verbose)

In [ ]:
## Check 1: equilibrium input data the same

print(hash2(rnn_dat['Et']))
print(hash2(rnn_dat['x_train']))
print(hash2(rnn_dat['y_train']))

In [ ]:
model_predict = train_rnn(
    rnn_dat,
    rnn_dat['hours'],
    activation=['linear','linear'],
    hidden_units=6,
    dense_units=1,
    dense_layers=1,
    verbose = False
)

In [ ]:
m = rnn_predict(model_predict, rnn_dat, rnn_dat['hours'], verbose = verbose)
case_data['m'] = m
note = 'm replaced by a solution from fmda_rnn_rain'
if 'note' in case_data:
    case_data['note'] = case_data['note'] + '\n' + note
else:
    case_data['note'] = note
check_data(case_data)

In [ ]:
plot_data(case_data,title2='with trained RNN',hmin=0,hmax=600)


In [ ]:
mse_data(case_data)

In [ ]:
plot_data(case_data,title2='RNN prediction',hmin=300,hmax=600)

In [ ]:
print(model_predict.get_weights())

In [ ]:
plot_model(model_predict)

---
---

## Model Validation

For a given RNN architecture, we train the model (<mark>from scratch each time for now</mark>)

In [ ]:
def run_rnn_test(d):
    # Helper function to deploy RNN for multiple cases
    # Given input dict, build rnn data, train model, and predcit
    # Return: model output vector (ndarray)
    
    rnn_dat = create_rnn_data(d) # format data
    model_predict = train_rnn(
        rnn_dat,
        rnn_dat['hours'],
        activation=['linear','linear'],
        hidden_units=6,
        dense_units=1,
        dense_layers=1,
        verbose = False
    )
    m = rnn_predict(model_predict, rnn_dat, rnn_dat['hours'], verbose = verbose)

    return m

In [ ]:
err_dict = {} # dictionary to save validation error for cases

In [ ]:
%%capture cap --no-stderr

## Loop through dictionary and run RNN on cases
for key in [*test_dict.keys()]:
    print(key, ':', test_dict[key]['title'])
    
    dict1 = test_dict[key]
    
    
    m = run_rnn_test(dict1)
    
    
    dict1['m']=m
    
    errs = mse_data(dict1)
    
    print('-'*25)
    
    err_dict[key] = {
        'title' : test_dict[key]['title'],
        'train' : errs[0],
        'test' : errs[1],
        'm_hash' : int(hash2(m)),
        'm' : m
    }

In [ ]:
# Print above output for visualization, save to text file

print(cap.stdout)

with open('errors_RNN1.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
## Save Validation Outputs as pickle file with all data

with open('errors_RNN1.pickle', 'wb') as handle:
    pickle.dump(err_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)